<a href="https://colab.research.google.com/github/budennovsk/Pandas/blob/master/testovoe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy as np
import os

# Функция для преобразования сферического изображения в перспективное
def transform_pano_to_perspective(image_path, output_dir, output_name):
    # Загрузка изображения
    img = cv2.imread(image_path)

    # Определение размеров выходного изображения
    height, width, _ = img.shape
    print(height, width,)
    fov = 90  # Угол обзора
    aspect_ratio = width / height

    # Определение матрицы камеры
    K = np.array([[width / (2 * np.tan(fov / 360 * np.pi)), 0, width / 2],
                  [0, width / (2 * np.tan(fov / 360 * np.pi)), height / 2],
                  [0, 0, 1]])

    # Определение источника преобразования
    dst_img = cv2.warpPerspective(img, K, (width, height))

    # Сохранение результата
    output_path = os.path.join(output_dir, output_name)
    cv2.imwrite(output_path, dst_img)
    print(f"Сохранено: {output_path}")

# Папка с изображениями и папка для выходных данных
input_dir = 'site_capture'
output_dir = 'perspective_images'
os.makedirs(output_dir, exist_ok=True)

# Преобразование всех изображений
for i in range(1, 5):
    input_path = os.path.join(input_dir, f"/content/{i}.jpg")
    transform_pano_to_perspective(input_path, output_dir, f"perspective_{i}.jpg")


3264 6528
Сохранено: perspective_images/perspective_1.jpg
3264 6528
Сохранено: perspective_images/perspective_2.jpg
3264 6528
Сохранено: perspective_images/perspective_3.jpg
3264 6528
Сохранено: perspective_images/perspective_4.jpg


In [9]:
import os
import shutil

# Папки для изображений
input_dir = '/content/perspective_images'
output_dir = '/content/openMVG_output'
database_path = '/content/database/database.db'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.dirname(database_path), exist_ok=True)

# Копирование изображений
shutil.copytree(input_dir, os.path.join(output_dir, 'images'))

# Запуск COLMAP команд
# 1. Создание базы данных
!colmap database_creator --database_path $database_path

# 2. Извлечение признаков
!colmap feature_extractor --database_path $database_path --image_path $output_dir/images

# 3. Сопоставление признаков
!colmap exhaustive_matcher --database_path $database_path

# 4. Построение модели
!colmap mapper --database_path $database_path --image_path $output_dir/images --output_path $output_dir/model



/bin/bash: line 1: colmap: command not found
/bin/bash: line 1: colmap: command not found
/bin/bash: line 1: colmap: command not found
/bin/bash: line 1: colmap: command not found


In [2]:
import cv2
import numpy as np
import os
import csv

# Папка с изображениями
# input_dir = '/content/perspective_images'
input_dir = '/content' # Замените на нужный путь к вашей папке с изображениями
output_ply = "output.ply"
output_csv = "/content/camera_trajectory.csv"

# Считываем изображения с помощью заданного цикла
images = []
for i in range(1, 5):  # Предполагаем, что изображения называются 1.jpg, 2.jpg, 3.jpg и 4.jpg
    input_path = os.path.join(input_dir, f"{i}.jpg")
    if os.path.exists(input_path):
        images.append(input_path)
    else:
        print(f"Изображение {input_path} не найдено.")

if not images:
    raise Exception("Нет изображений для обработки.")

# Инициализация детектора ORB и матчера
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

camera_poses = []  # Для хранения позиций камеры
point_cloud = []   # Для хранения 3D точек

# Параметры камеры
focal_length = 6.7  # Фокусное расстояние в мм
sensor_width = 36.0  # Ширина сенсора в мм
image_width = 11968  # Ширина изображения

# Вычисляем фокусное расстояние в пикселях
focal_px = (focal_length / sensor_width) * image_width

# Матрица калибровки камеры
K = np.array([[focal_px, 0, image_width / 2],
              [0, focal_px, image_width / 2],
              [0, 0, 1]])

for i in range(len(images) - 1):
    img1 = cv2.imread(images[i], 0)
    img2 = cv2.imread(images[i + 1], 0)

    # Извлечение признаков
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # Поиск соответствий
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Получение координат соответствующих точек
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Восстановление позы
    E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)

    # Добавление первой позы камеры
    if len(camera_poses) == 0:
        camera_poses.append((np.eye(3), np.zeros((3, 1))))

    prev_R, prev_t = camera_poses[-1]
    curr_R = R @ prev_R
    curr_t = prev_t + prev_R @ t
    camera_poses.append((curr_R, curr_t))

    # Триангуляция
    P1 = K @ np.hstack((prev_R, prev_t))
    P2 = K @ np.hstack((curr_R, curr_t))
    pts4D = cv2.triangulatePoints(P1, P2, pts1, pts2)
    pts4D /= pts4D[3]  # Нормализация
    point_cloud.append(pts4D[:3].T)

# Сохранение облака точек в формате PLY
with open(output_ply, 'w') as f:
    f.write("ply\nformat ascii 1.0\n")
    f.write(f"element vertex {sum(len(pc) for pc in point_cloud)}\n")
    f.write("property float x\nproperty float y\nproperty float z\n")
    f.write("end_header\n")
    for pc in point_cloud:
        for p in pc:
            f.write(f"{p[0]} {p[1]} {p[2]}\n")

# Сохранение траектории камеры в формате CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Index', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ'])
    for i, (R, t) in enumerate(camera_poses):
        rvec, _ = cv2.Rodrigues(R)
        writer.writerow([i, t[0][0], t[1][0], t[2][0], rvec[0][0], rvec[1][0], rvec[2][0]])


with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Index', 'X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ'])
    for i, (R, t) in enumerate(camera_poses):
        rvec, _ = cv2.Rodrigues(R)
        writer.writerow([i, t[0][0], t[1][0], t[2][0], rvec[0][0], rvec[1][0], rvec[2][0]])

print(f"Траектория камеры сохранена в {output_csv}")


Траектория камеры сохранена в /content/camera_trajectory.csv


In [4]:
import cv2
import numpy as np
import os
import csv

# Параметры камеры
focal_length = 6.7  # Фокусное расстояние в мм
sensor_width = 36.0  # Ширина сенсора в мм
image_width = 11968  # Ширина изображения

# Вычисляем фокусное расстояние в пикселях
focal_px = (focal_length / sensor_width) * image_width

# Матрица калибровки камеры
K = np.array([[focal_px, 0, image_width / 2],
              [0, focal_px, image_width / 2],
              [0, 0, 1]])

# Папка с изображениями
input_dir = '/content/'
output_dir = '/content/images_with_trajectory'
os.makedirs(output_dir, exist_ok=True)

# Чтение траектории камеры
camera_poses = []
with open("/content/camera_trajectory.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        t = np.array([[float(row['X'])], [float(row['Y'])], [float(row['Z'])]])
        rvec = np.array([float(row['RotX']), float(row['RotY']), float(row['RotZ'])])
        R, _ = cv2.Rodrigues(rvec)
        camera_poses.append((R, t))

# Чтение изображений и отображение траектории
for i in range(1, 5):
    img_path = os.path.join(input_dir, f"{i}.jpg")
    img = cv2.imread(img_path)

    if img is None:
        print(f"Изображение {img_path} не найдено.")
        continue

    height, width, _ = img.shape

    # Проецируем 3D-точки камеры на 2D пространство изображения
    for j, (R, t) in enumerate(camera_poses):
        # Координаты центра камеры в 3D пространстве
        camera_center = -R.T @ t
        camera_center_2D, _ = cv2.projectPoints(camera_center.T, np.zeros((3,1)), np.zeros((3,1)), K, None)
        x, y = camera_center_2D.ravel()

        # Рисуем точку на изображении
        cv2.circle(img, (int(x), int(y)), 5, (0, 0, 255), -1)

        # Нанесение текста с номером кадра
        cv2.putText(img, f"Cam {j}", (int(x) + 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Сохранение изображения с траекторией
    output_img_path = os.path.join(output_dir, f"trajectory_{i}.jpg")
    cv2.imwrite(output_img_path, img)
    print(f"Сохранено изображение с траекторией: {output_img_path}")


Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_1.jpg
Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_2.jpg
Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_3.jpg
Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_4.jpg


In [6]:
import cv2
import numpy as np
import os
import csv

# Параметры камеры
focal_length = 6.7  # Фокусное расстояние в мм
sensor_width = 36.0  # Ширина сенсора в мм
image_width = 11968  # Ширина изображения

# Вычисляем фокусное расстояние в пикселях
focal_px = (focal_length / sensor_width) * image_width

# Матрица калибровки камеры
K = np.array([[focal_px, 0, image_width / 2],
              [0, focal_px, image_width / 2],
              [0, 0, 1]])

# Папка с изображениями
input_dir = '/content'
output_dir = '/content/images_with_trajectory'
os.makedirs(output_dir, exist_ok=True)

# Чтение траектории камеры
camera_poses = []
with open("/content/camera_trajectory.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        t = np.array([[float(row['X'])], [float(row['Y'])], [float(row['Z'])]])
        rvec = np.array([float(row['RotX']), float(row['RotY']), float(row['RotZ'])])
        R, _ = cv2.Rodrigues(rvec)
        camera_poses.append((R, t))

# Чтение изображений и отображение траектории
for i in range(1, 5):
    img_path = os.path.join(input_dir, f"{i}.jpg")
    img = cv2.imread(img_path)

    if img is None:
        print(f"Изображение {img_path} не найдено.")
        continue

    height, width, _ = img.shape

    # Проецируем 3D-точки камеры на 2D пространство изображения
    for j, (R, t) in enumerate(camera_poses):
        # Координаты центра камеры в 3D пространстве
        camera_center = -R.T @ t
        camera_center_2D, _ = cv2.projectPoints(camera_center.T, np.zeros((3,1)), np.zeros((3,1)), K, None)
        x, y = camera_center_2D.ravel()

        # Рисуем красную точку на изображении
        cv2.circle(img, (int(x), int(y)), 5, (0, 0, 255), -1)

        # Нанесение текста с номером кадра
        cv2.putText(img, f"Cam {j}", (int(x) + 10, int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Сохранение изображения с траекторией
    output_img_path = os.path.join(output_dir, f"trajectory_{i}.jpg")
    cv2.imwrite(output_img_path, img)
    print(f"Сохранено изображение с траекторией: {output_img_path}")


Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_1.jpg
Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_2.jpg
Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_3.jpg
Сохранено изображение с траекторией: /content/images_with_trajectory/trajectory_4.jpg
